<a href="https://colab.research.google.com/github/AlexandriaVA-Energy-Office/dominion-interval-data/blob/main/KW_IntervalData_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is to be used for cleaning multiple column-organized interval data tables, transforming them, and appending them into a single, row-organized, batched import file for upload into EnergyCAP. It is built to accommodate both historic, inconsistently formatted files as well as raw, newly downloaded files from Dominion*.

This script is intended for demand/kW data files.

(*historic files will be read in as .xlsx, while newly downloaded files read in as .csv; need to adjust commented code accordingly when reading in file)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# All uploaded files, installed packages, variables, etc. clear after 90 min. idle time or 12 hours of use

import pandas as pd
from google.colab import drive
from google.colab import files
import os

appended_data=pd.DataFrame()
path='/content/drive/MyDrive/KW_IntDataFiles'

for WorkingFile in os.listdir(path):
# Use below two lines if this is historic file, remove other "intdata_OG" line.    
    #xlsx = pd.ExcelFile(f"{path}/{WorkingFile}")
    #intdata_OG = pd.read_excel(xlsx,'history-kw')
# read in raw csv file into new dataframe, setting standard column length at 51. No header, assuming raw file.
    intdata_OG = pd.read_csv(f"{path}/{WorkingFile}",header=None,names=list(range(51)))
# use os.rename function to rename each raw file to its account number, for easy future reference.
    source = path+'/'+WorkingFile
    newname = str(intdata_OG.iat[0,0])
    destination = path+'/'+newname+".csv"
    os.rename(source,destination)
# drop columns after 48 h-h increments;remove empty rows
    stripcolumns = intdata_OG.iloc[:,0:51]
    newfile = stripcolumns.dropna(axis=0,how='all')
# add column headers to raw file; does not affect files with existing header
    newfile.columns = ["account number","Recorder ID","Date","12:30 AM","1:00 AM","1:30 AM","2:00 AM","2:30 AM","3:00 AM","3:30 AM","4:00 AM","4:30 AM",
                      "5:00 AM","5:30 AM","6:00 AM","6:30 AM","7:00 AM","7:30 AM", "8:00 AM","8:30 AM","9:00 AM","9:30 AM","10:00 AM","10:30 AM","11:00 AM",
                      "11:30 AM","12:00 PM","12:30 PM","1:00 PM","1:30 PM","2:00 PM","2:30 PM","3:00 PM","3:30 PM","4:00 PM","4:30 PM","5:00 PM","5:30 PM",
                      "6:00 PM","6:30 PM","7:00 PM","7:30 PM","8:00 PM","8:30 PM","9:00 PM","9:30 PM","10:00 PM","10:30 PM","11:00 PM","11:30 PM","12:00 AM"]
# convert column ordered data into rows, adding new "Time" and "Demand" columns
    reformatted = pd.melt(newfile,id_vars=["account number","Recorder ID", "Date"],
                          value_vars=["12:30 AM","1:00 AM","1:30 AM","2:00 AM","2:30 AM","3:00 AM","3:30 AM","4:00 AM",
                                      "4:30 AM","5:00 AM","5:30 AM","6:00 AM",
                                      "6:30 AM","7:00 AM","7:30 AM", "8:00 AM",
                                      "8:30 AM","9:00 AM","9:30 AM","10:00 AM","10:30 AM","11:00 AM","11:30 AM","12:00 PM",
                                      "12:30 PM","1:00 PM","1:30 PM","2:00 PM","2:30 PM","3:00 PM","3:30 PM","4:00 PM",
                                      "4:30 PM","5:00 PM","5:30 PM","6:00 PM","6:30 PM","7:00 PM","7:30 PM","8:00 PM","8:30 PM",
                                      "9:00 PM","9:30 PM","10:00 PM","10:30 PM","11:00 PM","11:30 PM","12:00 AM"],
                          var_name="Time",value_name="Demand")
# Add new "Use" column; switch column order; drop "Recorder ID" column; delete blank "account number" rows    
    reformatted["Demand"] = reformatted["Demand"].apply(pd.to_numeric,errors='coerce').fillna(0)
    reformatted["Use"] = reformatted["Demand"] * 0.5
    column_titles = ["account number","Recorder ID","Date","Time","Demand","Use"]
    reformatted = reformatted.reindex(columns=column_titles)
    reformatted1 = reformatted.drop(["Recorder ID"], axis=1)
    finaltable = reformatted1.dropna(axis=0,subset=['account number'])
# Remove timestamp from date; change account number from float to int; convert non-numeric values to 0
    finaltable['Date'] = pd.to_datetime(finaltable['Date']).dt.date
    finaltable['account number'] = finaltable['account number'].astype(int)
    appended_data=appended_data.append(finaltable)
    print(f"{WorkingFile}")
appended_data.to_csv('/content/drive/MyDrive/KW_IntDataFiles/EnergyCAPupload.csv',index=False) 
